# Exercise 6. ICT Project: Communication Services and Security
### Cèsar Fernàndez Camón

#### Authors:
- Albert Pérez Datsira


## Problem 2
Let’s assume a CCMP (Cipher Block Chaining Message) with 8 bits length block. The counter blocks are computed
as follows:
- ctr(0) = key >> 2
- ctr(i) = ctr(i - 1) >> 2 ⊕ 0x53, i > 0

where (>>) is a circular shift to the right and (⊕) is a XOR operation. Consider:
* key = 0x22
* information = 0x0001020304
* AES as a XOR operation

1. Probe that the ciphered information is **0xAA522FB151** and its corresponding MIC is **0x26**.

2. Check that the same schema also works as a decipher.

## Utils
First of all, will be introduced the modules and functions provided used throughout the code, to compute the solution.

We are using `prettytable` to print more structured the results and you may install it by executing

`$ pip install prettytable`

or using any other package manager such as `Anaconda` by `$conda install -c conda-forge prettytable`

In [1]:
from prettytable import PrettyTable

### Circular shift right
This function performs a bitwise rotation assuming that the bits are rotated as if the left and right ends of the register were joined (circular ratation).

In [2]:
def circShift(val):
    tmp = val & shift_by
    return (val >> shift_by) | (tmp << (block_length - shift_by))

### XOR operation (⊕)

In [3]:
def xor(a, b):
    return a ^ b

### CTR (AES block cipher mode)


In [4]:
def ctr(val):
    ctr = []
    for i in range(0, 5):
        if (i == 0):
            tmp = circShift(circShift(val))
        else:
            tmp = xor(circShift(circShift(ctr[i - 1])), x) # XOR operation (⊕)
        ctr.append(tmp)
    return ctr

In [5]:
def bold(str):
    return "\033[1m{0}\033[0m".format(str)

## Problem execution

### Input data
Defining the variables with the statement data, and then printing formatted on bits results.

In [6]:
block_length = 8
x = 0x53

key = 0x22
information = 0x0001020304
information_array = [0x00, 0x01, 0x02, 0x03, 0x04]

ciphered_information = 0xAA522FB151
MIC = 0x26

shift_by = 1
IV = int('00000000', 2)

In [7]:
inputData = PrettyTable(title="Input Data", field_names=["Field", "Value"])

inputData.align["Field"] = "l"
inputData.align["Value"] = "r"

inputData.add_row([bold("Key"), key])
inputData.add_row([bold("Information"), information])
inputData.add_row([bold("Information array"), information_array])
inputData.add_row([bold("MIC"), MIC])
inputData.add_row([bold("Ciphered info"), ciphered_information])
inputData.add_row([bold("IV"), IV])
inputData.add_row([bold("x"), x])
inputData.add_row([bold("Block length"), block_length])
inputData.add_row([bold("Shift by"), shift_by])
print(inputData)


+-------------------------------------+
|              Input Data             |
+-------------------+-----------------+
| Field             |           Value |
+-------------------+-----------------+
| Key               |              34 |
| Information       |        16909060 |
| Information array | [0, 1, 2, 3, 4] |
| MIC               |              38 |
| Ciphered info     |    731523297617 |
| IV                |               0 |
| x                 |              83 |
| Block length      |               8 |
| Shift by          |               1 |
+-------------------+-----------------+


First, we musth have in mind the CCM schema,


<div style="width: 560px; margin-top: 10px;"><img src='assets/ccm_schema.png' alt='CCM Schema'/></div>

As can be seen though, to cypher the information the CTR should be applied, but to get the MIC the CBC takes part.

Besides, this case is based on 8 bits instead of 128 bits, 

therefore, the CCMP will have 8 bits and our block of information 40 bits, meaning 5 control blocks must be computed.

### Calculation CTR

In [8]:
CTR = ctr(key)
print(CTR)
for v in CTR: print("{0:08b} ".format(v), end='')

[136, 113, 15, 144, 119]
10001000 01110001 00001111 10010000 01110111 

### Data cyphering


Applying the CTR and the AES key into a XOR operation, the result of which will reach another XOR operation with each block of 8 bits the corresponding data following the below schema.

<div style="width: 580px; margin-top: 20px;"><img src='assets/ctr_mode.png' alt='CTR mode'/></div>

In [9]:
cyphered = []

[cyphered.append(xor(information_array[i], xor(CTR[i], key))) for i in range(len(CTR))]

print(cyphered)
for v in cyphered: print("{0:08b} ".format(v), end='')

[170, 82, 47, 177, 81]
10101010 01010010 00101111 10110001 01010001 

Thus, the cyphered data obtained results in ** **

## MIC calculation (checksum)

Now we must calculate, following the CCM Schema, the MIC applying the CBC mode schema

<div style="width: 580px; margin-top: 30px;"><img src='assets/cbc_mode.png' alt='CBC Mode' /></div>

taking into account both the IV and the key in the computation. But, the result for the last block should be equal to the desired MIC.

In [10]:
MIC = []

MIC.append(xor(xor(information_array[0], IV), key)) # first bloc appending
[MIC.append(xor(xor(information_array[i], MIC[i - 1]), key)) for i in range(1, len(information_array))]

for v in MIC: print("{0:08b} ".format(v), end='')  

00100010 00000001 00100001 00000000 00100110 

Thus, the fifth block (MIC) is

In [11]:
print("{0} = {1:08b} = {2}".format(bold("MIC"), MIC[4], hex(MIC[4])))

MIC = 00100110 = 0x26


## Part 2

Check that the same schema also works as a decyhper

To do this, we will use the cyphered information obtained before


## Data decyphering
Applying an XOR operation between the cyphered data, the CTR and the AES key.

In [12]:
decyphered = []

for i in range(len(CTR)): # applying the same method XOR based
    tmp = xor(cyphered[i], xor(CTR[i], key))
    decyphered.append(tmp)

for v in decyphered: print("{0:08b} ".format(v), end='')    

00000000 00000001 00000010 00000011 00000100 

## Result comparison
Once all the computations are done, let's focus on what results were obtained since the aim is to get as descyphered the same original information provided as input data by using the same methodology.

In [13]:
def toBin(val, str=''):
    for v in val:
        str += "{0:08b} ".format(v)
    return str

In [14]:
tableResult = PrettyTable(title="Results", field_names=["Field", "Value"])

tableResult.align["Field"] = "l"
tableResult.align["Value"] = "r"

tableResult.add_row([bold("Original information"), bold(toBin(information_array))])
tableResult.add_row([bold("CTR"), toBin(CTR)])
tableResult.add_row([bold("Cyphered"), toBin(cyphered)])
tableResult.add_row([bold("MIC"), toBin(MIC)])
tableResult.add_row([bold("Decyphered"), bold(toBin(decyphered))])
print(tableResult)

+----------------------------------------------------------------------+
|                               Results                                |
+----------------------+-----------------------------------------------+
| Field                |                                         Value |
+----------------------+-----------------------------------------------+
| Original information | 00000000 00000001 00000010 00000011 00000100  |
| CTR                  | 10001000 01110001 00001111 10010000 01110111  |
| Cyphered             | 10101010 01010010 00101111 10110001 01010001  |
| MIC                  | 00100010 00000001 00100001 00000000 00100110  |
| Decyphered           | 00000000 00000001 00000010 00000011 00000100  |
+----------------------+-----------------------------------------------+


Both the original and the deciphered results show the same values so it is safe to assume that the whole process has been performed correctly.